In [1]:
# CIFAR-10を読み込んで標準化を含めた前処理を行う
import numpy as np
from keras.datasets import cifar10
from keras.utils import np_utils

# CIFAR-10データを読み込む
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# 画像データをfloat32(浮動小数点数)型に変換
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# 訓練用とテスト用の画像データを標準化する
# 4次元テンソルのすべての軸方向に対して平均、標準偏差を求めるので
# axis=(0,1,2,3)は省略してもよい
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
# 標準化する際に標準偏差に極小値を加える
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

# 分類先のクラスの数
num_classes = 10
# 正解ラベルをワンホット表現に変換
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

Using TensorFlow backend.


In [3]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers
from keras.callbacks import LearningRateScheduler
from keras.optimizers import Adam

# ハイパーパラメーターの値を設定
weight_decay = 1e-4

# CNNを構築
model = Sequential()

# (第1層)畳み込み層1 正則化を行う
model.add(
    Conv2D(
        filters=32,                    # フィルターの数は32
        kernel_size=(3,3),             # 3×3のフィルターを使用
        input_shape=x_train.shape[1:], # 入力データの形状
        padding='same',                # ゼロパディングを行う 
        kernel_regularizer=regularizers.l2(weight_decay),
        activation='relu'              # 活性化関数はReLU
        ))
# 正規化
model.add(BatchNormalization())

# (第2層)畳み込み層2　正則化を行う
model.add(
    Conv2D(filters=32,                 # フィルターの数は32
           kernel_size=(3,3),          # 3×3のフィルターを使用
           padding='same',             # ゼロパディングを行う 
           kernel_regularizer=regularizers.l2(weight_decay),
           activation='relu'           # 活性化関数はReLU
           ))
# 正規化
model.add(BatchNormalization())

# (第3層)プーリング層1：ウィンドウサイズは2×2
model.add(
    MaxPooling2D(pool_size=(2,2)))
# ドロップアウト1：ドロップアウトは25％
model.add(Dropout(0.25))
 
# (第4層)畳み込み層3　正則化を行う
model.add(
    Conv2D(filters=32,                 # フィルターの数は32
           kernel_size=(3,3),          # 3×3のフィルターを使用
           padding='same',             # ゼロパディングを行う 
           kernel_regularizer=regularizers.l2(weight_decay),
           activation='relu'           # 活性化関数はReLU
           ))

# 正規化
model.add(BatchNormalization())

# (第5層)畳み込み層3　正則化を行う
model.add(
    Conv2D(filters=32,                 # フィルターの数は32
           kernel_size=(3,3),          # 3×3のフィルターを使用
           padding='same',             # ゼロパディングを行う 
           kernel_regularizer=regularizers.l2(weight_decay),
           activation='relu'           # 活性化関数はReLU
           ))
# 正規化
model.add(BatchNormalization())

# (第6層)プーリング層2：ウィンドウサイズは2×2
model.add(
    MaxPooling2D(pool_size=(2,2)))
# ドロップアウト2：ドロップアウトは25％
model.add(Dropout(0.25))
 
# (第6層)畳み込み層4　正則化を行う
model.add(
    Conv2D(filters=128,                # フィルターの数は128
           kernel_size=(3,3),          # 3×3のフィルターを使用
           padding='same',             # ゼロパディングを行う 
           kernel_regularizer=regularizers.l2(weight_decay),
           activation='relu'           # 活性化関数はReLU
          ))
# 正規化
model.add(BatchNormalization())

# (第7層)畳み込み層5　正則化を行う
model.add(
    Conv2D(filters=128,                # フィルターの数は128
           kernel_size=(3,3),          # 3×3のフィルターを使用
           padding='same',             # ゼロパディングを行う 
           kernel_regularizer=regularizers.l2(weight_decay),
           activation='relu'           # 活性化関数はReLU
          ))
# 正規化
model.add(BatchNormalization())

# (第8層)プーリング層3：ウィンドウサイズは2×2
model.add(
    MaxPooling2D(pool_size=(2,2)))
# ドロップアウト2：ドロップアウトは50％
model.add(Dropout(0.5))
 
# Flatten層　4階テンソルから2階テンソルに変換
model.add(Flatten())

# （第9層）出力層
model.add(
    Dense(num_classes,           # 出力層のニューロン数はnum_classes
          activation='softmax')) # 活性化関数はソフトマックス

# Sequentialオブジェクトのコンパイル
model.compile(
    loss='categorical_crossentropy', # 損失の基準は交差エントロピー誤差
    # 最適化アルゴリズムにAdamを使用
    optimizer=Adam(),
    metrics=['accuracy'] # 学習評価として正解率を指定
)

In [4]:
# モデルのサマリを表示
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_7 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_8 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 16, 16, 32)        9248      
__________

In [6]:
# 学習を行う

batch_size = 64 # ミニバッチのサイズ
epochs = 30     # 学習回数

history = model.fit(x_train,               # 訓練データ
                    y_train,               # 正解ラベル 
                    batch_size=batch_size, # 勾配計算に用いるミニバッチの数 
                    epochs=epochs,         # 学習を繰り返す回数
                    verbose=1,             # 学習の進捗状況を出力する
                    validation_data=(
                        x_test, y_test     # テストデータの指定
                    ))

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 620s 12ms/step - loss: 1.9728 - acc: 0.4112 - val_loss: 1.4362 - val_acc: 0.5166
Epoch 2/30
50000/50000 [==============================] - 597s 12ms/step - loss: 1.3349 - acc: 0.5716 - val_loss: 1.0307 - val_acc: 0.6527
Epoch 3/30
50000/50000 [==============================] - 558s 11ms/step - loss: 1.0969 - acc: 0.6462 - val_loss: 0.8955 - val_acc: 0.7053
Epoch 4/30
50000/50000 [==============================] - 560s 11ms/step - loss: 0.9417 - acc: 0.6952 - val_loss: 0.8083 - val_acc: 0.7368
Epoch 5/30
50000/50000 [==============================] - 556s 11ms/step - loss: 0.8483 - acc: 0.7238 - val_loss: 0.7858 - val_acc: 0.7443
Epoch 6/30
50000/50000 [==============================] - 544s 11ms/step - loss: 0.7809 - acc: 0.7475 - val_loss: 0.7515 - val_acc: 0.7609
Epoch 7/30
50000/50000 [==============================] - 544s 11ms/step - loss: 0.7416 - acc: 0.7620 - val_loss: 0.